In [1]:
import time, sys, os, subprocess, pickle, shutil, itertools, string, warnings
import numpy as np
from scipy.stats import pearsonr
from cClasses import cClassOne
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")


import networkx as nx
from sklearn.cluster import SpectralClustering
from scipy.sparse import csgraph

from sknetwork.clustering import Louvain, modularity
from sknetwork.linalg import normalize
from sknetwork.utils import bipartite2undirected, membership_matrix

ARG = 7402

df_fnanme = f'mod_big.txt'
T_ms = 14000000
stride_ms = 1000
dt = 0.01

set_evolved_weights = True


U = 0.2
SET = 0

NE = 2500
NI = 500


df = pd.read_pickle('df.pkl')

# the order of keys IS IMPORTANT for the cClasses not to break down
params = {
    "alpha": 50.0,    # Degree of log-STDP (50.0)
    "usd": 0.1,       # Release probability of a synapse (0.05 - 0.5)
    "JEI": 0.15,      # 0.15 or 0.20

    "T": 1800*1000.0,   # simulation time, ms
    "h": 0.01,          # time step, ms ??????

    # probability of connection
    "cEE": 0.2, # 
    "cIE": 0.2, #
    "cEI": 0.5, #
    "cII": 0.5, #

    # Synaptic weights
    "JEE": 0.15, #
    "JEEinit": 0.16, # ?????????????
    "JIE": 0.15, # 
    "JII": 0.06, #
    
    #initial conditions of synaptic weights
    "JEEh": 0.15, # Standard synaptic weight E-E
    "sigJ": 0.3,  #

    "Jtmax": 0.25, # J_maxˆtot
    "Jtmin": 0.01, # J_minˆtot # ??? NOT IN THE PAPER

    # Thresholds of update
    "hE": 1.0, # Threshold of update of excitatory neurons
    "hI": 1.0, # Threshold of update of inhibotory neurons

    "IEex": 2.0, # Amplitude of steady external input to excitatory neurons
    "IIex": 0.5, # Amplitude of steady external input to inhibitory neurons
    "mex": 0.3,        # mean of external input
    "sigex": 0.1,      # variance of external input

    # Average intervals of update, ms
    "tmE": 5.0,  #t_Eud EXCITATORY
    "tmI": 2.5,  #t_Iud INHIBITORY
    
    #Short-Term Depression
    "trec": 600.0,     # recovery time constant (tau_sd, p.13 and p.12)
    "Jepsilon": 0.001, # ????????
    
    # Time constants of STDP decay
    "tpp": 20.0,  # tau_p
    "tpd": 40.0,  # tau_d
    "twnd": 500.0, # STDP window lenght, ms
    
    "g": 1.25,        # ??????
    
    #homeostatic
    "itauh": 100,       # decay time of homeostatic plasticity, (100s)
    "hsd": 0.1,
    "hh": 10.0,  # SOME MYSTERIOUS PARAMETER
    "Ip": 1.0, # External current applied to randomly chosen excitatory neurons
    "a": 0.20, # Fraction of neurons to which this external current is applied
    
    "xEinit": 0.02, # the probability that an excitatory neurons spikes at the beginning of the simulation
    "xIinit": 0.01, # the probability that an inhibitory neurons spikes at the beginning of the simulation
    "tinit": 100.00, # period of time after which STDP kicks in (100.0)
    "U": 0.6,
    "taustf": 200,
    "taustd": 500,
    "Cp": 0.01875,
    "Cd": 0.0075,
    "HAGA": True,
    "asym": True,
    "stimIntensity": 0.55} 


params['asym'] = 0
params['HAGA'] = 1   
params['JEE'] = 0.15

params["U"] = df.iloc[ARG]['U']
params['g'] = 2.5
params["tinit"] = 100
params["JEEinit"] = 0.16
params["Cp"] = df.iloc[ARG]['Cp']
params["Cd"] = df.iloc[ARG]['Cd']
params["tpp"] = df.iloc[ARG]['tpp']
params["tpd"] = df.iloc[ARG]['tpd']
params["taustf"] = df.iloc[ARG]['taustf']
params["taustd"] = df.iloc[ARG]['taustd']
params["alpha"] = 50.00
params["itauh"] = 100


params["stimIntensity"] = 0.28

In [2]:
# instantiate model
m = cClassOne(NE, NI, ARG)

# set parameters
m.setParams(params)

# get parameters
ret = m.getState()

In [10]:
# simulate a given number of time steps

%%time
m.sim(int(stride_ms//dt))

CPU times: user 11.9 s, sys: 68.5 ms, total: 12 s
Wall time: 12.4 s


In [11]:
# dump spike states to files

m.dumpSpikeStates()

In [9]:
# toggle saving spike times (1 for on, 0 for off)
m.saveSpikes(1)